# Introduction: Deploy Anyscale Ray on AWS EC2 Instances

© 2025, Anyscale. All Rights Reserved

This notebook serves as a guide for deploying Anyscale on AWS EC2 instances using the custom **`anyscale cloud register`** method. It walks through the necessary steps from prerequisites to Ray installation using Anyscale.

Use it as a starting point and replace all placeholders (e.g.&nbsp;`{ANYSCALE_CLOUD_NAME}`) with values from your environment.

### Prerequisites

Before we begin, ensure you have the following tools installed:

```bash
# Install AWS CLI (version 2.15.0+)
# https://docs.aws.amazon.com/cli/latest/userguide/getting-started-install.html

# Configure AWS credentials
# https://docs.aws.amazon.com/cli/latest/userguide/cli-chap-configure.html

# Install Terraform (version 1.9+)
# https://developer.hashicorp.com/terraform/install

# Install Anyscale CLI (version 0.5.86+)
# https://docs.anyscale.com/reference/quickstart-cli/
```

You also need:
- An existing AWS Account
- Proper IAM permissions


<div class="alert alert-block alert-info">
<b>Alternative Terraform Installation:</b> If you are not able to install <b>Terraform 1.9+</b> with homebrew, you can try to install it with <code>tfenv</code>.

<details>
<summary>Click to expand installation steps</summary>

```bash
brew install tfenv
tfenv install 1.9.0
tfenv use 1.9.0
terraform version
```

</details>
</div>

## 1. Create Anyscale Resources with Terraform

Set up the necessary Terraform variables and apply the configuration:

### Creating Anyscale Resources

Steps for deploying Anyscale resources via Terraform:

* Review and modify [variables.tf](variables.tf) with your configurations
* (Optional) Create a `terraform.tfvars` file to override any defaults
* View [main.tf](main.tf) to see how the resources are created


In [ ]:
"""
Set the global variables for the deployment, which are the same as in the variables.tf file.
"""

ANYSCALE_CLOUD_NAME = "anyscale-cloud-ec2-private-example" # Replace with your actual Anyscale cloud name
AWS_REGION = "us-west-2"  # Replace with your actual AWS region
ANYSCALE_S3_BUCKET_NAME = "anyscale-ec2-new-us-west-2" # You can leave it empty for now, you can get it from the output of the terraform apply command, then you can populate it here.

In [ ]:
# Run Terraform commands

# Initialize Terraform
!terraform init

# Preview the changes
!terraform plan

# Apply the changes
!terraform apply -auto-approve


<details>
<summary>Sometimes, your AWS account may have too many existing resources consuming capacity, causing you to hit service limits. In such cases, it's a good idea to release unused resources—such as unassociated Elastic IPs (EIPs) or idle NAT gateways.
</summary>
During you installing, you may need to release the unused EIPs and deleted unused NAT.

To find the unattached EIPs:
```bash
aws ec2 describe-addresses | jq '.Addresses[] | select(.InstanceId == null and .NetworkInterfaceId == null)'
```
You will see a list of unattached EIPs.

To release one:

```bash
aws ec2 release-address --allocation-id eipalloc-xxxxxx
```

To find the unused NAT in your region, for example us-west-2:
```bash
aws ec2 describe-nat-gateways --region us-west-2 --filter "Name=state,Values=available" | jq '.NatGateways | length'
```

To delete one:

```bash
aws ec2 delete-nat-gateway --nat-gateway-id nat-xxxxxxxx
```

To identify VPCs that look like they might be safe to delete (test/development ones):
```bash
aws ec2 describe-vpcs --query 'Vpcs[*].[VpcId,Tags[?Key==`Name`].Value|[0]]' --output table | grep -E "(test|temp|dev|scratch|derp|floral|scrumptious)"
```

to delete one:
```bash
aws ec2 delete-vpc --vpc-id vpc-0f8bb12ddf9a451e9  # derp-vpc-derp-sunbeam
```

You may need to delete more depends on your scenario.

</detail>

<div class="alert alert-block alert-info">
<b>Take a note to the output of terraform apply! </b>You will need it when you register the Anyscale cloud to your cloud provider.
</div>

<details>
<summary>Sample output</summary>
```
Apply complete! Resources: 50 added, 0 changed, 0 destroyed.

Outputs:

anyscale_registration_command = <<EOT
anyscale cloud register --provider aws \
  --name anyscale-cloud-ec2-private-example \
  --region us-west-2 \
  --vpc-id vpc-0xxxxxxxxx \
  --subnet-ids subnet-xxxxxxxxxx,subnet-xxxxxxxxxxxx,subnet-0xxxxxxxxxx,subnet-0xxxxxxxx \
  --s3-bucket-id anyscale-ec2-new-us-west-2 \
  --anyscale-iam-role-id arn:aws:iam::959243851260:role/anyscale-ec2-role-new \
  --instance-iam-role-id arn:aws:iam::959243851260:role/anyscale-cluster-node-xxxxxxxxxxxx \
  --security-group-ids sg-xxxxxxxxx \
  --efs-id fs-xxxxxxxxxx

EOT
efs_id = "fs-0bd7bde3f2c914b3e"
instance_profile_arn = "arn:aws:iam::xxxxx:instance-profile/anyscale-cluster-node-xxxxxxxxxx"
s3_bucket_id = "anyscale-ec2-new-us-west-2"
security_group_id = "sg-xxxxxxxxxx"
subnet_ids = [
  "subnet-0a000000000000000",
  "subnet-03000000000000000",
  "subnet-00000000000000000",
  "subnet-0a000000000000000",
]
vpc_id = "vpc-0000000000000"```
</details>

## 2. Register the Anyscale Cloud

First, ensure you're logged into Anyscale. if you cannot run it in console, please run it in your local terminal:

In [ ]:
# First, ensure you're logged into Anyscale
!anyscale login

Then use the output of `terraform apply` to register Anyscale cloud. You only need to replace the `--name` parameter with your preferred `ANYSCALE_CLOUD_NAME`. The command looks like:

```bash
anyscale cloud register ...
```

You will get output like:

```text
(anyscale +45.3s) Successfully created cloud anyscale-cloud-ec2-private-example, and it's ready to use.
```

## 3. Test

Once the clusterOnce the cluster is created, you can test it by submitting a job from your terminal: is created, you can test it by submitting a job:


In [ ]:
!cd ../test && python test_job.py --cloud-name {ANYSCALE_CLOUD_NAME} --stack-type vm

# You can check the job status by running:
!anyscale job list --cloud {ANYSCALE_CLOUD_NAME}

You just start a job and you can see the logs from your Anyscale Console. You can view the running results from Anyscale console in "Jobs".

## 4. Cleanup

When you're done, you can terminate the cluster and clean up resources:


In [ ]:
# Unregister the cloud
!anyscale cloud delete {ANYSCALE_CLOUD_NAME}

# Clean up S3 bucket if you cannot delete it because it is not empty, then run "terraform destroy --auto-approve" again
!aws s3 rm s3://{ANYSCALE_S3_BUCKET_NAME} --recursive

# Clean up Terraform resources
!terraform destroy --auto-approve

# Verify no Anyscale clouds are registered
!anyscale cloud list

# Verify Terraform resources are destroyed
!terraform show

## 5. Conclusion

You have now successfully set up the Anyscale environment on AWS EC2 instances. This includes:

1. Creating the necessary AWS resources using Terraform:
   - VPC and subnets
   - IAM roles and instance profiles
   - S3 bucket for storage
   - EFS for shared file storage
   - Security groups
2. Registering the Anyscale Cloud
3. Creating and connecting to a Ray cluster
4. Running a sample Ray job

Additionally, you've learned about optional advanced configurations:
- Autoscaling with Ray's built-in autoscaler
- Adding load balancing with AWS Application Load Balancer
- Using GPU instances for accelerated computing

You can now use this environment to run Ray workloads on Anyscale using EC2 instances.
